In [1]:
from DemoServerDB import ConnectDatabase
from mapping import doc

In [2]:
temp = ConnectDatabase()

In [3]:
temp.conn

<connection object at 0x7faa104ce398; dsn: 'dbname=postgresdemotwo user=clientdemo password=xxxxxxxxxxx host=postgresdemotwo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com port=5432 sslmode=require', closed: 0>

In [4]:
doc['column_map']['TRANSACTION_MASTER']

{'category': 'category',
 'cust_id': 'id',
 'product_id': 'prod_id',
 'quantity': 'purchasequantity',
 'timestamp': 'date'}

In [7]:
query = 'SELECT "%s","%s","%s" FROM "%s"'%(doc['column_map']['TRANSACTION_MASTER']['cust_id'],
                                      doc['column_map']['TRANSACTION_MASTER']['category'],
                                      doc['column_map']['TRANSACTION_MASTER']['timestamp'],
                                      doc['table_map']['TRANSACTION_MASTER'])

In [8]:
print query

SELECT "id","category","date" FROM "trans"


In [9]:
temp.conn.rollback()

In [10]:
temp.cur.execute(query)

In [11]:
col_names = [desc[0] for desc in temp.cur.description]

In [12]:
col_names

['id', 'category', 'date']

In [13]:
import pandas as pd

In [14]:
trans_table = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [ ]:
trans_table.head()

In [15]:
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'}, inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['category']:'category'}, inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['timestamp']:'timestamp'}, inplace=True)

In [16]:
trans_table.head()

,cust_id,category,timestamp
0,555549182,5613,2012-10-21
1,555549182,5307,2012-10-21
2,555549182,2108,2012-10-21
3,555549182,6315,2012-10-21
4,555549182,5827,2012-10-21


In [17]:
trans_table.shape

(71932603, 3)

In [20]:
type(trans_table.timestamp[0])

str

In [22]:
trans_table['timestamp'] = pd.to_datetime(trans_table['timestamp'])

In [25]:
type(trans_table.timestamp[0])

pandas.tslib.Timestamp

In [26]:
group_data = trans_table.groupby(['cust_id','category']).size().reset_index().rename(columns = {0:'counter'})

In [27]:
group_data.head()

,cust_id,category,counter
0,100010021,201,1
1,100010021,311,1
2,100010021,410,1
3,100010021,416,8
4,100010021,417,1


In [28]:
repeated_data = group_data.query('counter != 1')

In [29]:
repeated_data.shape

(7569745, 3)

In [31]:
xyz = trans_table.merge(repeated_data, on=['cust_id', 'category'], how='left')

In [32]:
xyz.shape

(71932603, 4)

In [33]:
xyz.head()

,cust_id,category,timestamp,counter
0,555549182,5613,2012-10-21,18.0
1,555549182,5307,2012-10-21,18.0
2,555549182,2108,2012-10-21,25.0
3,555549182,6315,2012-10-21,95.0
4,555549182,5827,2012-10-21,19.0


In [44]:
pd.set_option('precision', 0)

In [45]:
xyz.head()

,cust_id,category,timestamp,counter
0,555549182,5613,2012-10-21,18
1,555549182,5307,2012-10-21,18
2,555549182,2108,2012-10-21,25
3,555549182,6315,2012-10-21,95
4,555549182,5827,2012-10-21,19


In [55]:
def x(frame):
    print frame.timestamp.index.min()
    print frame.timestamp.index.max()

In [ ]:
xyz.groupby(['cust_id','category']).apply(x)